# Optimiser Config Report

Continuing the previous section, let's now look at the `config.json` and `report.json` files generated by the solvers.

In [1]:
import json
from IPython.display import display, JSON

with open('../4.2_optimiser_solver/sa/config.json') as f:
    config = json.load(f)
display(JSON(config))

<IPython.core.display.JSON object>

The `config.json` file is a checkpoint for the accelerator configuration. It can be passed to [`fpgaconvnet-hls`](https://github.com/AlexMontgomerie/fpgaconvnet-hls) to generate the accelerator implementation. In addition, users can load this checkpoint and apply it to an [`Network`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/network/Network.py) instance with the [`prototxt_to_fpgaconvnet`](https://github.com/AlexMontgomerie/fpgaconvnet-model/tree/dev-petros/fpgaconvnet/parser#L356) function.

In [2]:
with open('../4.2_optimiser_solver/sa/report.json') as f:
    report = json.load(f)
display(JSON(report))

<IPython.core.display.JSON object>

In [3]:
from fpgaconvnet.optimiser.solvers import Solver
from fpgaconvnet.parser.Parser import Parser
from fpgaconvnet.platform.Platform import Platform

onnx_path = "../3.1_model_onnx_parser/fp16/vgg16_bn.onnx"
parser = Parser(custom_onnx=True, batch_size=1)
net = parser.onnx_to_fpgaconvnet(onnx_path)
net = parser.prototxt_to_fpgaconvnet(net, '../4.2_optimiser_solver/sa/config.json')

platform = Platform()
platform.update("../../fpgaconvnet-optimiser/examples/platforms/zcu104.toml")

solver = Solver(net, platform)
solver.solver_status()

CRITICAL WARNING: node Flatten_31 is skipped in hardware
| COST:                 |    | RESOURCES:   |           |            |                 |                 |            |            |            |            |
|-----------------------|----|--------------|-----------|------------|-----------------|-----------------|------------|------------|------------|------------|
|                       |    | URAM         | BRAM      | DSP        | LUT             | FF              | BW         | BW_IN      | BW_OUT     | BW_WEIGHT  |
| 0.694833 (throughput) |    | 0.00/96      | 97.14/624 | 70.79/1728 | 49193.07/230400 | 22436.00/460800 | 2.12/153.6 | 2.05/153.6 | 0.07/153.6 | 0.00/153.6 |
|                       |    | 0.00 %       | 15.57 %   | 4.10 %     | 21.35 %         | 4.87 %          | 1.38 %     | 1.34 %     | 0.05 %     | 0.00 %     |



The `report.json` file provides the detailed breakdown for the resource and performance estimations. For example, you can get the system latency and throughput by accessing:

In [4]:
print("Latency: ", report['network']['performance']['latency (s)'])
print("Throughput: ", report['network']['performance']['throughput (FPS)'])

Latency:  1.50997847
Throughput:  0.6622610983320841


In terms of resources, since we are looking at a single device setup where partitions are executed in a time-multiplexed manner, `avg_resource_usage` is the metric of interest.

In [5]:
print("Resource: ", report['network']['avg_resource_usage'])

Resource:  {'LUT': 49193.07142857143, 'FF': 22436.0, 'BRAM': 97.14285714285714, 'DSP': 70.78571428571429, 'URAM': 0.0}
